In [ ]:
# スクレイピングに必要なユーザー名、パスワード、URLなどを消去しているので動きません

In [1]:
import pandas as pd
import random
import time

In [2]:
# モジュール読み込み
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By

In [3]:
# ランキングを読み込み
ranking = pd.read_csv('', encoding='utf-8-sig', index_col=0)

In [4]:
log_in_url = ""
username = ''
password = ''

In [5]:
# chromedriver の path
CHROMEDRIVER = r""
# Chrome起動
chrome_service = service.Service(executable_path=CHROMEDRIVER)
browser = webdriver.Chrome(service=chrome_service)
browser.implicitly_wait(3)
# urlにアクセス
browser.get(log_in_url)
# ログイン
elem = browser.find_element(by='id', value='_username')
elem.send_keys(username)
elem = browser.find_element(by='id', value='_password')
elem.send_keys(password)

log_in = browser.find_element(by='id', value='log_in')
log_in.click()

In [8]:
# idごとに口コミを取得する関数
# ブラウザはもう開いている
def make_answers_csv(m_id):
    
    # 口コミ URL の基本形
    # 新しい順にしている
    answer_url_base = f''

    # ページ数
    page_num = 1 
    # 最大ページ数
    max_page = 15


    # 口コミの情報を格納するリスト
    answers = []

    # 最後のページ（または max_page）まで繰り返す
    while page_num<max_page:
        # ページ数を加える
        answer_url = answer_url_base + str(page_num)

        # アクセス
        browser.get(answer_url)
        # 次のアクセスの前にスリープ
        time.sleep(random.uniform(1, 3))

        # 口コミの要素を取得
        elems = browser.find_elements(by='class name', value='article')

        # 各口コミごとに情報を格納
        for elem in elems:
            answer_info_list = elem.text.split('\n')

            title = answer_info_list[1]

            date = pd.to_datetime(answer_info_list[2],
                                  format='回答日%Y年%m月%d日')

            user = answer_info_list[3].replace('回答者 ', '')

            score = float(answer_info_list[4])

            content = answer_info_list[5:-4]

            answers.append([title, date, user, score, content])

        # 最後のページなら終了
        if browser.find_elements(by='class name', value='paging_link-more')[-1].text != '次へ':
            break

        page_num += 1
        
    df_answers = pd.DataFrame(answers)
    df_answers.columns = ['title', 'date', 'user', 'score', 'content']
    df_answers['m_id'] = m_id
    
    # 結果をcsvに書き出す
    df_answers.to_csv(f'company_answers/answers_{m_id}.csv', encoding='utf-8-sig')

In [ ]:
for m_id in ranking['m_id'][:100]:
    try:
        make_answers_csv(m_id)
    except:
        pass